# Melody Generation with Markov Chains

In [1]:
import sys
sys.path.append("..") 

import matplotlib.pyplot as plt
import music21 as m21
import torch
from preprocess import load_songs_in_kern, NoteEncoder, TERM_SYMBOL

In [ ]:
torch.manual_seed(0);

In [ ]:
encoder = NoteEncoder()
scores = load_songs_in_kern('./../deutschl/erk')
enc_songs = encoder.encode_songs(scores)

In [ ]:
scores[0].show('mid')

In [ ]:
' '.join(enc_songs[0])

In [ ]:
print(f'longest melody: {max(len(m) for m in enc_songs)}')
print(f'shortest melody: {min(len(m) for m in enc_songs)}')

In [ ]:
symbols = sorted(list(set([item for sublist in 
                           enc_songs for item in sublist])))
stoi = {s:i+1 for i, s in enumerate(symbols)}
stoi[TERM_SYMBOL] = 0
itos = {i: s for s, i in stoi.items()}
print(f'n_symbols: {len(itos)}')

In [ ]:
N = torch.zeros((len(stoi), len(stoi)))
for enc_song in enc_songs:
    chs = [TERM_SYMBOL] + enc_song + [TERM_SYMBOL]
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        N[ix1, ix2] += 1

In [ ]:
print(
    f'There are {len(N[N > 0.0])/len(stoi)**2 *100} % non-zero entries')

In [ ]:
P = N.float()
P = P / P.sum(dim=1, keepdim=True)

In [ ]:
plt.rcParams.update({'font.size': 6})
plt.figure(figsize=(8, 8))
plt.imshow(P, cmap='Blues')

In [ ]:
n_scores = 5
generated_encoded_songs = []
for _ in range(n_scores):
    generated_encoded_song = []
    char = '.'
    while True:
        ix = torch.multinomial(P[stoi[char]], num_samples=1, replacement=True).item()
        char = itos[ix]
        if char == '.':
            break
        generated_encoded_song.append(char)
        #break
    len(generated_encoded_song)
    generated_encoded_songs.append(generated_encoded_song)

In [ ]:
generated_songs = list(map(lambda generated_encoded_song: encoder.decode_song(generated_encoded_song), generated_encoded_songs))
generated_songs[4].show('mid')

In [ ]:
log_likelyhood = 0.0
n = 0
for m in enc_songs:
    chs = ['.'] + m + ['.']
    for ch1, ch2 in zip(chs, chs[1:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        prob = P[ix1, ix2]
        logprob = torch.log(prob)
        log_likelyhood += logprob
        n += 1
        #print(f'{ch1}->{ch2}: {prob:.4f}')

print(f'{log_likelyhood=}')
nll = -log_likelyhood
print(f'avg negative log likelyhood: {(nll/n)}')